# 数据扩增

对训练集进行扩增，验证集和测试集不要动

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
import warnings
warnings.filterwarnings('ignore')
import re, jieba, requests, json, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('../data/TrainSet.csv')
train['content'] = train['content'].astype(str)
print('train has {} rows and {} columns'.format(train.shape[0], train.shape[1]))
train.head()

train has 330 rows and 2 columns


,content,class
0,为了保障网络安全，维护网络空间主权和国家安全、社会公共利益，保护公民、法人和其他组织的合法权...,法规目的
1,在中华人民共和国境内建设、运营、维护和使用网络，以及网络安全的监督管理，适用本法。,职责区分
2,国家坚持网络安全与信息化发展并重，遵循积极利用、科学发展、依法管理、确保安全的方针，推进网络...,法规倡议
3,国家制定并不断完善网络安全战略，明确保障网络安全的基本要求和主要目标，提出重点领域的网络安全...,法规倡议
4,国家采取措施，监测、防御、处置来源于中华人民共和国境内外的网络安全风险和威胁，保护关键信息基...,职责区分


In [98]:
def translate(word):
    '''访问有道词典 api'''
    url = 'http://fanyi.youdao.com/translate?smartresult=dict&smartresult=rule&smartresult=ugc&sessionFrom=null'
    # 传输的参数，其中 i 为需要翻译的内容
    key = {
        'type': "AUTO",
        'i': word,
        "doctype": "json",
        "version": "2.1",
        "keyfrom": "fanyi.web",
        "ue": "UTF-8",
        "action": "FY_BY_CLICKBUTTON",
        "typoResult": "true"
    }
    # key 这个字典为发送给有道词典服务器的内容
    response = requests.post(url, data=key)
    # 判断服务器是否响应成功
    if response.status_code == 200:
        # 返回响应的结果
        return response.text
    else:
        print("有道词典调用失败")
        # 响应失败就返回空
        return None

def get_main(word):
    '''如果输入不只一句话，则将翻译过的字符串拼接起来'''
    list_trans = translate(word)
    result = json.loads(list_trans)
    stri = ''
    for i in result['translateResult'][0]:
        stri += i['tgt']
    return stri

def translate_reverse(x):
    '''反向翻译'''
    x = get_main(x)
    time.sleep(0.1) # 程序休眠0.1
    return get_main(x)

In [4]:
# 去除空格等
train['content'] = train['content'].apply(lambda x: x.replace('\u3000','') \
                                                      .replace('\n','') \
                                                      .replace('\r','') \
                                                      .strip()) 
train['content'].head(15)

0     为了保障网络安全，维护网络空间主权和国家安全、社会公共利益，保护公民、法人和其他组织的合法权...
1             在中华人民共和国境内建设、运营、维护和使用网络，以及网络安全的监督管理，适用本法。
2     国家坚持网络安全与信息化发展并重，遵循积极利用、科学发展、依法管理、确保安全的方针，推进网络...
3     国家制定并不断完善网络安全战略，明确保障网络安全的基本要求和主要目标，提出重点领域的网络安全...
4     国家采取措施，监测、防御、处置来源于中华人民共和国境内外的网络安全风险和威胁，保护关键信息基...
5     国家倡导诚实守信、健康文明的网络行为，推动传播社会主义核心价值观，采取措施提高全社会的网络安...
6     国家积极开展网络空间治理、网络技术研发和标准制定、打击网络违法犯罪等方面的国际交流与合作，推...
7     国家网信部门负责统筹协调网络安全工作和相关监督管理工作。国务院电信主管部门、公安部门和其他有...
8     网络相关行业组织按照章程，加强行业自律，制定网络安全行为规范，指导会员加强网络安全保护，提高...
9     任何个人和组织使用网络应当遵守宪法法律，遵守公共秩序，尊重社会公德，不得危害网络安全，不得利...
10    国家支持研究开发有利于未成年人健康成长的网络产品和服务，依法惩治利用网络从事危害未成年人身心...
11    国家建立和完善网络安全标准体系。国务院标准化行政主管部门和国务院其他有关部门根据各自的职责，...
12    国务院和省、自治区、直辖市人民政府应当统筹规划，加大投入，扶持重点网络安全技术产业和项目，支...
13    国家推进网络安全社会化服务体系建设，鼓励有关企业、机构开展网络安全认证、检测和风险评估等安全服务。
14    国家鼓励开发网络数据安全保护和利用技术，促进公共数据资源开放，推动技术创新和经济社会发展。国...
Name: content, dtype: object

In [5]:
train.to_csv('../data/TrainSet0.csv',encoding='gb18030', index=False)

In [6]:
translate_reverse('在这场比赛中，您将可以访问结合了基因表达（特征）和细胞存活率（标签）数据的独特数据集。')

'在游戏中,你将能够访问(特性)结合基因表达和细胞生存独特的数据集(标签)的数据。'

In [11]:
train0 = train.copy()
start = time.time()
train0['content'] = train0['content'].apply(lambda x: translate_reverse(x))
print('This program costs {:.2f} seconds'.format(time.time()-start))
train0.head()

This program costs 154.21 seconds


,content,class
0,"为了保证网络安全,维护网络主权和国家安全、社会公共利益,保护公民的合法权益,法人和其他组织,...",法规目的
1,"中华人民共和国境内的施工、操作、维护和使用网络,以及网络安全的监督管理,适用本法。",职责区分
2,"国家坚持网络安全与信息技术的发展,积极利用、科学发展和管理,确保安全按照政策,促进网络基础设...",法规倡议
3,"国家发展和改善网络安全策略,明确网络安全的基本要求和主要目标,提出了关键领域的网络安全策略、...",法规倡议
4,"国家采取措施来监控、国防和处置来源于中华人民共和国和网络安全风险和威胁,保护关键信息基础设施...",职责区分


In [7]:
sample = train.iloc[17,0]
sample
#type(sample)   

'国家实行网络安全等级保护制度。网络运营者应当按照网络安全等级保护制度的要求，履行下列安全保护义务，保障网络免受干扰、破坏或者未经授权的访问，防止网络数据泄露或者被窃取、篡改：（一）制定内部安全管理制度和操作规程，确定网络安全负责人，落实网络安全保护责任；（二）采取防范计算机病毒和网络攻击、网络侵入等危害网络安全行为的技术措施；（三）采取监测、记录网络运行状态、网络安全事件的技术措施，并按照规定留存相关的网络日志不少于六个月；（四）采取数据分类、重要数据备份和加密等措施；（五）法律、行政法规规定的其他义务。'

In [12]:
train0.iloc[17,0] # 展示一下扩增后的例子

'国家实行系统的网络安全防护水平。网络运营商应当按照网络安全等级保护系统的要求,执行以下安全保护义务,保障对干扰,破坏或未经授权的访问,防止网络数据泄露或被盗,篡改:(1)建立内部安全管理制度和操作规程,识别、安全主管、落实责任保护网络安全;(2)以防止计算机病毒和网络攻击,网络入侵技术措施等危害网络安全的行为;(3)监测和记录网络运行状态,网络安全事件的技术方法,依照有关规定和留存有关web日志不少于6个月;(4)将数据分类、数据备份和加密措施;(5)法律、行政法规规定的其他义务。'

## 数据扩增计划

原始数据330行
+ 使用反向翻译以整句为单位将数据扩增一倍
+ 使用反向翻译(日文)以整句为单位将数据扩增一倍
+ 在分词后，将一些词语随机进行反向翻译
+ 在分词后，将一些词语随机替换为 噪音
+ 

In [13]:
train0.to_csv('../data/augment.csv',encoding='gb18030', index=False)

In [47]:
get_main('训练')

'training'

In [63]:
import urllib.request
import urllib.parse
import json
 
# 等待用户输入需要翻译的单词
#content = input('请输入需要翻译的单词：')
content = '疯狂的海鲜'
# 有道翻译的url链接
url = 'http://fanyi.youdao.com/translate?smartresult=dict&smartresult=rule&smartresult=ugc&sessionFrom=null'
 
# 发送给有道服务器的数据
data = {}
 
# 需要翻译的文字
data['i'] = content
# 下面这些都先按照我们之前抓包获取到的数据
data['from'] = 'zh'
data['to'] = 'fr'
data['smartresult'] = 'dict'
data['client'] = 'fanyideskweb'
data['salt'] = '1500349255670'
data['sign'] = '997742c66698b25b43a3a5030e1c2ff2'
data['doctype'] = 'json'
data['version'] = '2.1'
data['keyfrom'] = 'fanyi.web'
data['action'] = 'FY_BY_CL1CKBUTTON'
data['typoResult'] = 'true'
data['type'] = 'ZH_CN2fr'
# 对数据进行编码处理
data = urllib.parse.urlencode(data).encode('utf-8')
 
# 创建一个Request对象，把url和data传进去，并且需要注意的使用的是POST请求
request = urllib.request.Request(url=url, data=data, method='POST')
# 打开这个请求
response = urllib.request.urlopen(request)
# 读取返回来的数据
result_str = response.read().decode('utf-8')
# 把返回来的json字符串解析成字典
result_dict = json.loads(result_str)
 
# 获取翻译结果
print('翻译的结果是：%s' % result_dict)

翻译的结果是：{'type': 'ZH_CN2EN', 'errorCode': 0, 'elapsedTime': 6, 'translateResult': [[{'src': '疯狂的海鲜', 'tgt': 'Wild seafood'}]]}


In [100]:
def translate_ja(word):
    '''访问有道词典 api'''
    url = 'http://fanyi.youdao.com/translate?smartresult=dict&smartresult=rule&smartresult=ugc&sessionFrom=null'
    # 传输的参数，其中 i 为需要翻译的内容
    key = {
        'i': word,
        "doctype": "json",
        "version": "2.1",
        "keyfrom": "fanyi.web",
        "ue": "UTF-8",
        "action": "FY_BY_CLICKBUTTON",
       # "from": "zh",
        "to": "ja" ,
        "typoResult": "true",
        "type":"ZH_CN2JA"
    }
    # key 这个字典为发送给有道词典服务器的内容
    response = requests.post(url, data=key)
    # 判断服务器是否响应成功
    if response.status_code == 200:
        # 返回响应的结果
        return response.text
    else:
        print("有道词典调用失败")
        # 响应失败就返回空
        return None

def get_main_japan(word):
    '''如果输入不只一句话，则将翻译过的字符串拼接起来'''
    list_trans = translate_ja(word)
    result = json.loads(list_trans)
    stri = ''
    for i in result['translateResult'][0]:
        stri += i['tgt']
    return stri

def translate_reverse_ja(x):
    '''反向翻译'''
    x = get_main_japan(x)
    time.sleep(0.1) # 程序休眠0.1
    return get_main(x)

#get_main_japan('训练')
translate_reverse_ja('训练模型')

'训练模式'

In [99]:
translate_reverse_ja('国家实行网络安全等级保护制度。网络运营者应当按照网络安全等级保护制度的要求，履行下列安全保护义务，保障网络免受干扰、破坏或者未经授权的访问，防止网络数据泄露或者被窃取、篡改：（一）制定内部安全管理制度和操作规程，确定网络安全负责人，落实网络安全保护责任；（二）采取防范计算机病毒和网络攻击、网络侵入等危害网络安全行为的技术措施；（三）采取监测、记录网络运行状态、网络安全事件的技术措施，并按照规定留存相关的网络日志不少于六个月；（四）采取数据分类、重要数据备份和加密等措施；（五）法律、行政法规规定的其他义务。')

'国家网络安全保护等级制度的执行计划。如果网络运营者网络安全等级保护制度的要求,按照下面的安全保护义务的履行,保障网络,妨碍或破坏的授权访问,防止泄漏和推行网络数据被篡改,盗难:(1)制定内部的安全管理制度和指南,并确定网络安全负责人,保护网络安全责任落实;(2)防范计算机病毒、网络攻击、侵入等危害网络安全行为的技术措施;(3)进行监测,网络的状态,根据网络安全运行记录技术措施,按照事件的相关规定,大家的网络日志6个月不能少;(4)分类,重要数据的备份和数据加密等措施;(5)法律、行政法规规定的其他义务。'

In [97]:
get_main_japan('国家实行网络安全等级保护制度。网络运营者应当按照网络安全等级保护制度的要求，履行下列安全保护义务，保障网络免受干扰、破坏或者未经授权的访问，防止网络数据泄露或者被窃取、篡改')

'国家サイバー安全保護等級制度を施行する計画だ。インターネット運営者ならサイバー安全等級保護制度の要求どおり、下记の安全保護義務の履行を保障からネットワークを妨害したり、破壊の不用意授権訪問、流出防止やインターネットデータ改竄し、盗难されて'

In [7]:
from googletrans import Translator
translator = Translator(service_urls=["translate.google.cn"])

# 调用翻译函数，指定原语言的代码(en)，和目标语言的代码(zh-CN)


In [11]:
result = translator.translate('您好，我是大树。你个大傻逼。想不想喝星巴克。', src='zh-CN', dest='fr')
print(result.text)

Bonjour, je suis un grand arbre. Toi idiot. Voulez-vous boire du Starbucks?
